In [5]:
# Load model directly
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import set_seed
set_seed(999)

import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

f_name = 'IMDB_synthetic.csv'

import os
if f_name not in os.listdir('./'):
    f_ptr = open(f_name, 'w')
else:
    f_ptr = open(f_name, 'a')

# Set beam search parameters
num_beams = num_seqs = 10
top_k = 500
top_p = 0.7
div_pen = 4.0
repetition_penalty = 2.5
temperature = 1.0


def write_data(positive):
    p_dict = {True: "positive", False: "negative"}
    prompt = p_dict[positive] + ": This movie is"
    tokenizer = AutoTokenizer.from_pretrained("lvwerra/gpt2-imdb-pos")
    model = AutoModelForCausalLM.from_pretrained("lvwerra/gpt2-imdb-pos").to(device)
    input_ids = tokenizer.encode(prompt, return_tensors='pt').to(device)
    attention_mask = torch.ones(input_ids.shape, dtype=torch.long, device=input_ids.device)

    output = model.generate(input_ids, max_length=256, num_return_sequences=num_seqs, num_beams=num_beams, top_k=top_k, top_p=top_p, 
                            num_beam_groups=num_beams, diversity_penalty=div_pen, temperature=temperature, early_stopping=False, 
                            repetition_penalty=repetition_penalty, attention_mask=attention_mask)
    for i, review in enumerate(output):
        generated_review = tokenizer.decode(review, skip_special_tokens=True)[24:]
        # print(f"Generated review {i+1}: {generated_review}")
        f_ptr.write("\"%s\",%s\n" % (generated_review, p_dict[positive]))
    return

write_data(True)
write_data(False)

/home/zl310/anaconda3/envs/michael_ece684/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/home/zl310/anaconda3/envs/michael_ece684/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:396: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `500` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [1]:
print(len('Positive: This movie is '))

24
